In [4]:
import numpy as np
total_distance = np.load("/mnt/NAS2-2/data/SAD_gangnam_MDMR/distance/6mm/gangnam_total_distance.npy")
sad_distance = np.load("/mnt/NAS2-2/data/SAD_gangnam_MDMR/distance/6mm/gangnam_sad_distance.npy")

In [5]:
total_distance.shape

(20290, 117, 117)

In [7]:
sad_distance.shape

(20290, 60, 60)

In [8]:
import pandas as pd

In [37]:
total_code_list = pd.read_csv("../../input/gangnam_total_code_list.csv", header=None)
sad_code_list = pd.read_csv("../../input/gangnam_sad_code_list.csv", header=None)

## temporary
# Remove the first row (index 0) from each DataFrame
total_code_list = total_code_list.drop(0).reset_index(drop=True)
sad_code_list = sad_code_list.drop(0).reset_index(drop=True)


In [38]:
# Check the structure of the loaded DataFrames
print(total_code_list.head())  # Check the first few rows of the total_code_list
print(sad_code_list.head())    # Check the first few rows of the sad_code_list

       0
0  s0005
1  s0004
2  c0009
3  s0012
4  c0010
       0
0  s0005
1  s0004
2  s0012
3  s0008
4  s0019


In [83]:
# Assuming the first column contains the codes
mapping = sad_code_list[0].apply(lambda x: total_code_list[total_code_list[0] == x].index.tolist())

# Display the mapping
mapping = mapping.explode().reset_index()
mapping.columns = ['sad_code_index', 'total_code_index']
# 매핑 과정이 올바르게 수행되었는지 확인
print(mapping.head())

# 혹시 매핑되지 않은 값이 있는지 확인
unmapped_sad_codes = sad_code_list[0][~sad_code_list[0].isin(total_code_list[0])]
if not unmapped_sad_codes.empty:
    print(f"These codes from sad_code_list could not be mapped to total_code_list: {unmapped_sad_codes.values}")

   sad_code_index total_code_index
0               0                0
1               1                1
2               2                3
3               3                6
4               4                8


In [48]:
total_indices = mapping["total_code_index"].astype(int).values
total_indices

array([  0,   1,   3,   6,   8,  15,  16,  21,  27,  28,  31,  34,  40,
        41,  42,  45,  46,  48,  49,  50,  51,  52,  53,  54,  55,  56,
        57,  58,  59,  60,  61,  62,  63,  64,  65,  67,  71,  74,  77,
        83,  88,  89,  90,  91,  93,  94,  95,  97,  99, 100, 101, 102,
       103, 107, 108, 110, 111, 112, 115, 116])

In [91]:
new_sad_distance = total_distance[:, total_indices][:, :, total_indices].astype(np.float64).copy()
sad_distance = sad_distance.astype(np.float64).copy()

new_sad_distance = new_sad_distance.astype(np.float64)
sad_distance = sad_distance.astype(np.float64)

# 추출된 new_sad_distance의 모양이 올바른지 확인
print(f"new_sad_distance shape: {new_sad_distance.shape}")
print(f"sad_distance shape: {sad_distance.shape}")

# 두 distance 매트릭스가 동일한 형태인지 확인
if new_sad_distance.shape == sad_distance.shape:
    print("new_sad_distance has the correct shape.")
else:
    print("Shape mismatch between new_sad_distance and sad_distance.")

new_sad_distance shape: (20290, 60, 60)
sad_distance shape: (20290, 60, 60)
new_sad_distance has the correct shape.


In [94]:
# np.allclose에서 허용하는 상대 오차와 절대 오차를 설정
comparison = np.allclose(new_sad_distance, sad_distance, rtol=1e-7, atol=1e-7)

# 비교 결과 출력
if comparison:
    print("The new sad distance matrix matches the existing sad distance matrix for the first voxel.")
else:
    print("The new sad distance matrix does not match the existing sad distance matrix for the first voxel.")

The new sad distance matrix matches the existing sad distance matrix for the first voxel.
